In [15]:
import gradio as gr
import joblib
import librosa
import numpy as np

# Load model và scaler đã huấn luyện
model = joblib.load('svm_model_Music.pkl')
scaler = joblib.load('scaler_Music.pkl')

def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3.0)

    features = []

    # 1. length
    features.append(float(len(y)))

    # 2. Chroma STFT
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features.append(float(np.mean(chroma)))
    features.append(float(np.var(chroma)))

    # 3. RMS
    rms = librosa.feature.rms(y=y)
    features.append(float(np.mean(rms)))
    features.append(float(np.var(rms)))

    # 4. Spectral Centroid
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    features.append(float(np.mean(spec_cent)))
    features.append(float(np.var(spec_cent)))

    # 5. Spectral Bandwidth
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features.append(float(np.mean(spec_bw)))
    features.append(float(np.var(spec_bw)))

    # 6. Spectral Rolloff
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features.append(float(np.mean(rolloff)))
    features.append(float(np.var(rolloff)))

    # 7. Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    features.append(float(np.mean(zcr)))
    features.append(float(np.var(zcr)))

    # 8. Harmony (dùng harmonic energy, không phải raw signal)
    y_harmonic = librosa.effects.harmonic(y)
    features.append(float(np.mean(y_harmonic)))
    features.append(float(np.var(y_harmonic)))

    # 9. Percussive (perceived rhythm)
    y_percussive = librosa.effects.percussive(y)
    features.append(float(np.mean(y_percussive)))
    features.append(float(np.var(y_percussive)))

    # 10. Tempo
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    features.append(float(tempo))

    # 11. MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for coeff in mfcc:
        features.append(float(np.mean(coeff)))
        features.append(float(np.var(coeff)))

    return np.array(features)


def classify_music(audio_file):
    try:
        features = extract_features(audio_file)

        if scaler:
            features = scaler.transform(features.reshape(1, -1))
        else:
            features = features.reshape(1, -1)

        prediction = model.predict(features)[0]
        return f"🎧 **Thể loại dự đoán:** `{prediction}`"
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"

with gr.Blocks(theme=gr.themes.Soft()) as app:
    
    gr.Markdown("""
    # 🎵 **Music Genre Classifier**
    Upload To Predict to gener.

    Support Audio files such as `.wav`, `.mp3`  
    
    """)

    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="🎶 Chọn file âm thanh")
        output_text = gr.Markdown()

    with gr.Row():
        predict_button = gr.Button("Predict")

    predict_button.click(fn=classify_music, inputs=audio_input, outputs=output_text)

app.launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
